In [ ]:
import os
import ccxt
import schedule
import time
from tabulate import tabulate
from dotenv import load_dotenv
from datetime import datetime, timedelta
import ntplib
import numpy as np
import logging
import asyncio
import nest_asyncio
import threading
import csv
from ntplib import NTPClient  # Import NTPClient
import matplotlib.pyplot as plt
 
# Load environment variables from .env file
load_dotenv()
 
# Create an empty list to store trading signals
trading_signals = []
 
# Fetch API credentials from environment variables
API_KEY = os.getenv('API_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')
PASSPHRASE = os.getenv('PASSPHRASE')
 
# Initialize the KuCoin Futures exchange instance
exchange = ccxt.kucoinfutures({
    'apiKey': API_KEY,
    'secret': SECRET_KEY,
    'password': PASSPHRASE,
    'enableRateLimit': True  # Adjust as needed
})
      
# Create an NTP client instance
ntp_client = NTPClient()
 
# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Set your desired log level
    format="%(asctime)s [%(levelname)s] - %(message)s",
    handlers=[
        logging.FileHandler("trading_log.txt"),  # Log to a file
        logging.StreamHandler()  # Log to the console
    ]
)
 
# Create a logger instance
logger = logging.getLogger(__name__)
 
def calculate_atr(high_prices, low_prices, close_prices, period=19):
    # Calculate True Range (TR)
    tr = [max(hl, hc, lc) - min(hl, hc, lc) for hl, hc, lc in zip(high_prices, close_prices, low_prices)]
 
    # Calculate the Average True Range (ATR) using a period (e.g., 14)
    atr = np.mean(tr[-period:])
    return atr
 
def calculate_rsi(data, atr, period=19):
    # Calculate the differences between consecutive closing prices
    price_diff = np.diff(data)
 
    # Calculate the positive and negative price changes
    positive_changes = np.where(price_diff > 0, price_diff, 0)
    negative_changes = np.where(price_diff < 0, -price_diff, 0)
 
    # Smooth the average gains and losses using ATR
    smoothed_avg_gain = np.mean(positive_changes[-period:]) / atr
    smoothed_avg_loss = np.mean(negative_changes[-period:]) / atr
 
    # Calculate the relative strength (RS) and RSI
    rs = smoothed_avg_gain / smoothed_avg_loss if smoothed_avg_loss != 0 else 0
    rsi = 100 - (100 / (1 + rs))
 
    return rsi
 
def calculate_obv(close_prices, volume_data):
    obv = [0]  # Initialize OBV with 0
    for i in range(1, len(close_prices)):
        if close_prices[i] > close_prices[i - 1]:
            obv.append(obv[-1] + volume_data[i])  # Adding volume on up days
        elif close_prices[i] < close_prices[i - 1]:
            obv.append(obv[-1] - volume_data[i])  # Subtracting volume on down days
        else:
            obv.append(obv[-1])  # No change in OBV on unchanged days
    return obv
 
def calculate_smoothed_imbalance(data, alpha=0.1):
    smoothed_data = [data[0]]  # Initialize with the first data value
    for i in range(1, len(data)):
        smoothed_data.append(alpha * data[i] + (1 - alpha) * smoothed_data[i - 1])
    return smoothed_data
 
def fetch_ohlcv_and_analyze_order_book(symbol, depth=20, max_retries=3):
    retries = 0
    # Initialize a list to store historical imbalance percentages
    historical_imbalance_percentage = []
    historical_obv = []  # Initialize a list to store historical OBV values
   
    while retries < max_retries:
        try:
            # Use NTP to synchronize your system's time
            response = ntp_client.request('time.google.com')
            current_time = datetime.fromtimestamp(response.tx_time)
            
            # Fetch OHLCV data for ATR and TR calculation
            ohlcv_data = exchange.fetch_ohlcv(symbol, '5m')  # Adjust timeframe as needed
            close_prices = np.array([item[4] for item in ohlcv_data])
            high_prices = np.array([item[2] for item in ohlcv_data])
            low_prices = np.array([item[3] for item in ohlcv_data])
            
            # Fetch volume data
            volume_data = np.array([item[5] for item in ohlcv_data])
 
            # Calculate OBV
            obv = calculate_obv(close_prices, volume_data)
            
            # Calculate True Range (TR)
            tr = [max(hl, hc, lc) - min(hl, hc, lc) for hl, hc, lc in zip(high_prices, close_prices, low_prices)]
 
            # Calculate Average True Range (ATR) using a period (e.g., 14)
            atr = np.mean(tr[-14:])
            
            # Calculate RSI using close prices
            rsi = calculate_rsi(close_prices, atr)
 
            # Fetch the order book for the specified symbol and depth
            order_book = exchange.fetch_order_book(symbol, limit=20)
            bids = order_book['bids']
            asks = order_book['asks']
 
            # Extract bid prices and quantities
            bid_prices = [bid[0] for bid in bids]
            bid_quantities = [bid[1] for bid in bids]
 
            # Extract ask prices and quantities
            ask_prices = [ask[0] for ask in asks]
            ask_quantities = [ask[1] for ask in asks]
 
            # Create subplots for bids and asks
            plt.figure(figsize=(10, 4))
 
            # Plot bids in green
            plt.subplot(1, 2, 1)
            plt.barh(range(len(bid_prices)), bid_quantities, tick_label=bid_prices, color='green')
            plt.title('Top 20 Bids')
            plt.xlabel('Quantity')
            plt.ylabel('Price')
 
            # Plot asks in red
            plt.subplot(1, 2, 2)
            plt.barh(range(len(ask_prices)), ask_quantities, tick_label=ask_prices, color='red')
            plt.title('Top 20 Asks')
            plt.xlabel('Quantity')
            plt.ylabel('Price')
 
            # Show the plots
            plt.tight_layout()
            plt.show()
    
            # Calculate the total volume of bids and asks
            total_bids_volume = sum(bid[1] for bid in bids)
            total_asks_volume = sum(ask[1] for ask in asks)
 
            # Calculate the current order book imbalance percentage
            current_imbalance_percentage = ((total_bids_volume - total_asks_volume) / (total_bids_volume + total_asks_volume)) * 100
 
            # Calculate the weighted order book imbalance using the midpoint of bid and ask prices
            weighted_bids_volume = sum(bid[1] * (bid[0] + ask[0]) / 2 for bid, ask in zip(bids, asks))
            weighted_asks_volume = sum(ask[1] * (bid[0] + ask[0]) / 2 for bid, ask in zip(bids, asks))
 
            if weighted_bids_volume + weighted_asks_volume != 0:
                current_weighted_imbalance_percentage = (weighted_bids_volume - weighted_asks_volume) / (weighted_bids_volume + weighted_asks_volume) * 100
            else:
                current_weighted_imbalance_percentage = 0  # Default value when the total volume is zero
 
            # Print order book analysis results along with OBV
            obv_value = obv[-1]
            obv_interpretation = "Neutral"
            if obv_value > 0:
                obv_interpretation = "Bullish (Buyers Dominant)"
            elif obv_value < 0:
                obv_interpretation = "Bearish (Sellers Dominant)"
 
            print(
                f"Order Book Analysis for {symbol} - Imbalance: {current_imbalance_percentage:.2f}% (Weighted Imbalance: {current_weighted_imbalance_percentage:.2f}%) - RSI: {rsi:.2f} - OBV: {obv_value:.2f} ({obv_interpretation})")
 
            # Append the current imbalance percentage to the historical list
            historical_imbalance_percentage.append(current_imbalance_percentage)
            historical_obv.append(obv[-1])
 
            # Calculate smoothed order book imbalance using EMA
            smoothed_imbalance = calculate_smoothed_imbalance(historical_imbalance_percentage)
 
            # Generate trading signal and proposed entry price based on RSI and order book imbalance
            trading_signal, proposed_entry_price = generate_trading_signal(
                rsi,
                current_imbalance_percentage,
                close_prices,
                high_prices,  # Add this argument
                low_prices,   # Add this argument
                bids,
                asks
            )
            
            print("Trading Signal:", trading_signal)
            if proposed_entry_price:
                print("Proposed Entry Price:", proposed_entry_price)
            print("=" * 50)
           
            # Exit the retry loop if data is successfully fetched and analyzed
            break
           
        except Exception as e:
            retries += 1
            print(f"Error fetching or analyzing order book: {e}")
            print(f"Retrying... ({retries}/{max_retries})")
            time.sleep(10)  # Wait for 10 seconds before retrying
    
    # Return the calculated values
    return rsi, current_imbalance_percentage, close_prices, high_prices, low_prices, bids, asks
 
def generate_trading_signal(rsi, imbalance_percentage, close_prices, high_prices, low_prices, bids, asks):
    # Calculate the RSI divergence threshold (adjust as needed)
    rsi_divergence_threshold = 5
   
    # Calculate ATR using high_prices and low_prices
    atr = calculate_atr(high_prices, low_prices, close_prices)
   
    if imbalance_percentage >= 25:  # Positive imbalance condition
        # Check for bullish RSI divergence (oversold RSI)
        if rsi < 29:
            proposed_entry_price = bids[0][0]
            return "Validated Bullish Divergence (Long)", proposed_entry_price
    elif imbalance_percentage <= -24:  # Negative imbalance condition
        # Check for bearish RSI divergence (overbought RSI)
        if rsi > 69:
            proposed_entry_price = asks[0][0]
            return "Validated Bearish Divergence (Short)", proposed_entry_price
    else:
        # Check for hidden bullish divergence (higher low in RSI)
        if rsi < 30 and rsi + rsi_divergence_threshold < calculate_rsi(close_prices[-2:], atr):
            proposed_entry_price = bids[0][0]
            return "Hidden Bullish Divergence (Long)", proposed_entry_price
       
        # Check for hidden bearish divergence (lower high in RSI)
        if rsi > 70 and rsi - rsi_divergence_threshold > calculate_rsi(close_prices[-2:], atr):
            proposed_entry_price = asks[0][0]
            return "Hidden Bearish Divergence (Short)", proposed_entry_price
   
    return "No Entry", None
       
def create_order_with_fibonacci_levels(exchange, symbol, side, entry_price, leverage, amount):
    try:
        # Calculate Fibonacci retracement levels
        fibonacci_levels = [0.236, 0.382, 0.618]  # You can customize these levels
        
        # Calculate the price range between entry price and recent high/low
        ohlcv_data = exchange.fetch_ohlcv(symbol, '4h', limit=100)  # Adjust timeframe and limit as needed
        recent_high = max(item[2] for item in ohlcv_data)
        recent_low = min(item[3] for item in ohlcv_data)
        price_range = recent_high - recent_low
        
        # Calculate stop-loss and take-profit prices based on Fibonacci levels
        stop_loss_price = entry_price - price_range * fibonacci_levels[0]
        take_profit_price = entry_price + price_range * fibonacci_levels[-1]
 
        # Create the main limit order with leverage
        main_order = exchange.create_order(
            symbol,
            type='limit',
            side=side,
            amount=amount,
            price=entry_price,
            params={
                'postOnly': True,
                'timeInForce': 'GTC',
                'leverage': leverage
            }
        )
        print("Main Order Created:", main_order)
 
        # Create the stop-loss order
        stop_loss_order = exchange.create_order(
            symbol,
            type='limit',
            side='sell' if side == 'buy' else 'buy',
            amount=amount,
            price=stop_loss_price
        )
        print("Stop-Loss Order Created:", stop_loss_order)
 
        # Create the take-profit order
        take_profit_order = exchange.create_order(
            symbol,
            type='limit',
            side='sell' if side == 'buy' else 'buy',
            amount=amount,
            price=take_profit_price
        )
        print("Take-Profit Order Created:", take_profit_order)
 
        return main_order, stop_loss_order, take_profit_order
    except Exception as e:
        print(f"Error creating orders with Fibonacci retracement-based levels: {e}")
        return None, None, None
 
# Modify the execute_order_book_analysis function
def execute_order_book_analysis(symbol, leverage, amount):
    while True:
        
        # Fetch OHLCV data and analyze order book
        rsi, imbalance_percentage, close_prices, high_prices, low_prices, bids, asks = fetch_ohlcv_and_analyze_order_book(symbol)
       
        # Generate trading signal and proposed entry price based on RSI and order book imbalance
        trading_signal, proposed_entry_price = generate_trading_signal(
            rsi,
            imbalance_percentage,
            close_prices,
            high_prices,
            low_prices,
            bids,
            asks
        )
       
        # Get the current timestamp
        timestamp = int(time.time() * 1000)
   
        # Convert timestamp to datetime
        timestamp_datetime = datetime.fromtimestamp(timestamp / 1000.0)
 
        # Print the timestamp along with the trading signal in a human-readable format
        print("Timestamp:", timestamp_datetime)
        print("Trading Signal:", trading_signal)
        if proposed_entry_price:
            print("Proposed Entry Price:", proposed_entry_price)
        print("=" * 50)
        
        # Create a new signal dictionary
        new_signal = {
            "timestamp": timestamp_datetime,
            "trading_signal": trading_signal,
            "proposed_entry_price": proposed_entry_price
        }
 
        # Call the saving function to update the CSV file
        save_trading_signals_to_csv()
       
        # Inside execute_order_book_analysis
        if trading_signal != "No Entry" and proposed_entry_price:
            # Create a limit order based on the signal
            if trading_signal.startswith("Validated Bullish"):
                create_order_with_fibonacci_levels(exchange, symbol, 'buy', proposed_entry_price, leverage, amount)
            elif trading_signal.startswith("Validated Bearish"):
                create_order_with_fibonacci_levels(exchange, symbol, 'sell', proposed_entry_price, leverage, amount)

        # Create a new signal dictionary
        new_signal = {
            "timestamp": timestamp_datetime,
            "trading_signal": trading_signal,
            "proposed_entry_price": proposed_entry_price,
            "order_book_imbalance": imbalance_percentage,  # Include order book imbalance
            "rsi": rsi  # Include RSI
        }

        # Append the new signal to trading_signals
        trading_signals.append(new_signal)
       
        time.sleep(60)  # Sleep for 60 seconds before fetching again
 
# Save trading signals, order book imbalance, and RSI to a CSV file
def save_trading_signals_to_csv():
    with open("xrp_signal.csv", "w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(["Timestamp", "Trading Signal", "Proposed Entry Price", "Order Book Imbalance", "RSI"])
        
        for signal in trading_signals:
            timestamp = signal["timestamp"]
            trading_signal = signal["trading_signal"]
            proposed_entry_price = signal["proposed_entry_price"]
            order_book_imbalance = signal["order_book_imbalance"]  # Add this line
            rsi = signal["rsi"]  # Add this line
            
            csv_writer.writerow([timestamp, trading_signal, proposed_entry_price, order_book_imbalance, rsi])

 
# Call execute_order_book_analysis directly for your single symbol analysis
symbol_to_analyze = 'XRP/USDT:USDT'
leverage = 10  # Define your desired leverage
amount = 6000  # Define your desired amount
execute_order_book_analysis(symbol_to_analyze, leverage, amount)


In [3]:
%pip uninstall TA-Lib
%pip install TA-Lib



Found existing installation: TA-Lib 0.4.28
Uninstalling TA-Lib-0.4.28:
  Would remove:
    /Users/apple/anaconda3/lib/python3.11/site-packages/TA_Lib-0.4.28.dist-info/*
    /Users/apple/anaconda3/lib/python3.11/site-packages/talib/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
Y